In [ ]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Viridis256
from bokeh.models import ColorBar
from bokeh.transform import linear_cmap

In [ ]:
# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
#  Read the avocado data grouped by region with lat and Lon
df = pd.read_excel('Avocado.xlsx')
df['AveragePrice'] = df['AveragePrice'].round(2)
df.head()

In [ ]:
# Define coord as tuple (lat,long)
df['coordinates'] = list(zip(df['latitude'], df['longitude']))

# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coordinates'] ]

In [ ]:
# Create mercator column in df
df['mercator'] = mercators

# Split that column out into two separate columns - mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

df.head()

In [ ]:
# Select tile set to use
# Google maps requires an API key

chosentile = get_provider(Vendors.CARTODBPOSITRON)

In [ ]:
#  Add Price Scale for bubble size
scale = 20
df['PriceScale'] = df['AveragePrice'] * scale

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=df)

In [ ]:
#  Plot

# Set tooltips - these appear when we hover over a data point 
tooltips = [("Region","@Region"), ("Price","@AveragePrice")]

# Create figure
p = figure(title = 'Avocado Prices by region in the United States',
           x_axis_type="mercator", 
           y_axis_type="mercator", 
           tooltips = tooltips) 

#  No axis on map
p.axis.visible = False  

#  Dynamice width and height
p.sizing_mode = 'scale_both'

# Add map - place map in background
p.add_tile(chosentile)

# Add points using mercator coordinates
mapper = linear_cmap(field_name='AveragePrice', palette=Viridis256, low=min(df['AveragePrice']), high=max(df['AveragePrice']))

p.circle(x = 'mercator_x', 
         y = 'mercator_y', 
         color = mapper,
         source=source, 
         size='PriceScale')

#  Define and show the color scale bar
color_bar = ColorBar(color_mapper=mapper['transform'], width=20,  location=(0,0))
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('Avocado Bokeh.html', title='Avocado Prices by region in the United States')

In [ ]:
# Show map
show(p)